# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib notebook
import sys
###
#!pip install openslide-python
#https://github.com/FAU-DLM/python-wsi-preprocessing.git
###
sys.path.append('../preprocessing_pipeline/python-wsi-preprocessing/')
from wsi import slide, filter, tiles, util

import fastai
from fastai.vision import *

import pandas as pd
import numpy as np
import os
from pathlib import Path
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
from enum import Enum
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
import openslide
from openslide.lowlevel import *
Image.MAX_IMAGE_PIXELS = 10000000000

PATH = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome/')
FONT_PATH=PATH/'1984-Happines-Regular.ttf'
WSIS_CORTICOTROP = PATH/'corticotrop'
WSIS_GONADOTROP = PATH/'gonadotrop'
ROIS_CORTICOTROP = PATH/'corticotrop_ROIs'
ROIS_GONADOTROP = PATH/'gonadotrop_ROIs'
TILES_CORTICOTROP = PATH/'tiles_corticotrop_scoring_function_1_thresh_0.55'
TILES_GONADOTROP = PATH/'tiles_gonadotrop_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_2 = PATH/'tiles_corticotrop_2_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_2 = PATH/'tiles_gonadotrop_2_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_3 = PATH/'tiles_gonadotrop_3_scoring_function_1_thresh_0.55'

ROIS_EXPERIMENTING = PATH/'rois_experimenting'
TILES_EXPERIMENTING = PATH/'tiles_experimenting'

PATH_RELAPSE = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/')
RELAPSE_WSIS_EXPERIMENTING = PATH_RELAPSE/'wsis_experimenting'
RELAPSE_IMAGES_EXPERIMENTING = PATH_RELAPSE/'imgs_experimenting'
RELAPSE_TILES_EXPERIMENTING = PATH_RELAPSE/'tiles_experimenting'

NONE_RELAPSE_WSIS = PATH_RELAPSE/'wsis_non_relapse'
NONE_RELAPSE_IMAGES = PATH_RELAPSE/'imgs_non_relapse'
NONE_RELAPSE_TILES = PATH_RELAPSE/'tiles_non_relapse'

RELAPSE_WSIS = PATH_RELAPSE/'wsis_relapse'
RELAPSE_IMAGES = PATH_RELAPSE/'imgs_relapse'
RELAPSE_TILES = PATH_RELAPSE/'tiles_relapse'


def show(np):
    return util.np_to_pil(np)

Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

def show_multiple_images(path, rows = 3, figsize=(128, 64)):
    imgs = [open_image(p) for p in path.ls()]
    show_all(imgs=imgs, r=rows, figsize=figsize)
    
def show_multiple_images_big(path:pathlib.Path):
    for p in path.ls():
        plt.imshow(mpimg.imread(str(p)))
        plt.show()
        
def get_wsi_name_from_path(path):
    split = path.stem.split('-')
    return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}'

def get_roi_name_from_path(path):
        split = path.stem.split('-')
        if split[2] == 'HE':
                return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}-{split[4]}'
        else:
            return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}-{split[4]}-{split[5]}'
        
class AdenomaType(Enum):
    experimenting = 0
    corticotrop = 1
    gonadotrop = 2
    relapse_experimenting = 3
    relapse = 4
    non_relapse = 5

In [2]:
adenomaType = AdenomaType.gonadotrop

if adenomaType == AdenomaType.gonadotrop:
    rois_path = ROIS_GONADOTROP
    tiles_path = TILES_GONADOTROP
    tiles_path2 = TILES_GONADOTROP_2
    tiles_path3 = TILES_GONADOTROP_3
    
if adenomaType == AdenomaType.corticotrop:
    rois_path = ROIS_CORTICOTROP
    tiles_path = TILES_CORTICOTROP
    tiles_path2 = TILES_CORTICOTROP_2
    
if adenomaType == AdenomaType.experimenting:
    wsi_path = ROIS_EXPERIMENTING
    rois_path = ROIS_EXPERIMENTING
    tiles_path = TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse_experimenting:
    wsi_path = RELAPSE_WSIS_EXPERIMENTING
    rois_path = RELAPSE_IMAGES_EXPERIMENTING
    tiles_path = RELAPSE_TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse:
    wsi_path = RELAPSE_WSIS
    rois_path = RELAPSE_IMAGES
    tiles_path = RELAPSE_TILES  
    
if adenomaType == AdenomaType.non_relapse:
    wsi_path = NONE_RELAPSE_WSIS
    rois_path = NONE_RELAPSE_IMAGES
    tiles_path = NONE_RELAPSE_TILES

In [3]:
rois_path.mkdir(exist_ok=True)
tiles_path3.mkdir(exist_ok=True)

# Get tiles from ROIs no tiles have been extracted from yet

In [25]:
rois = list(set([get_roi_name_from_path(p) for p in rois_path.ls()]))

In [26]:
rois[0]

'606-14-III-HE-ROI_1-LH'

In [27]:
len(rois)

7796

In [28]:
ts = list(set([get_roi_name_from_path(p) for p in (tiles_path.ls()+tiles_path2.ls())]))

In [29]:
ts[0]

'606-14-III-HE-ROI_1-LH'

In [30]:
len(ts)

6640

In [33]:
rois_no_ts_extracted_yet = list(set(rois) - set(ts))

In [34]:
len(rois_no_ts_extracted_yet)

1156

In [48]:
ts = list(set([get_roi_name_from_path(p) for p in (tiles_path.ls()+tiles_path2.ls())]))
paths = []
for p in rois_path.ls():
    b = False
    for t in ts:
        if str(t) in str(p):
            b = True
            continue
    if(not b):
        paths.append(p)

In [49]:
len(paths)

1153

In [ ]:
tiles.WsiOrROIToTilesMultithreaded(paths, tiles_path3, 1024,1024, is_wsi=False)